In [4]:
REP_DIR = "/home/graffc/extra/fire_prediction/"
SRC_DIR = REP_DIR + 'src/'
DATA_DIR = REP_DIR + 'data/'

# Load system-wide packages
import os
import sys
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import cumfreq
from scipy.stats.stats import pearsonr
import datetime as dt
import pytz
import math
from IPython.display import HTML
%matplotlib inline

# Load project packages
os.chdir(SRC_DIR)
from features.loaders import load_modis_df, load_gfs_weather, load_cluster_df
from helper import date_util as du
from visualization.mapping import make_map
from helper.geometry import get_default_bounding_box
import helper.geometry as geo
from visualization.mapping import make_map
import visualization.plotting as plotv
#import models.train as trn
#from models import metrics
import visualization.mapping as mapv

plt.rcParams['figure.figsize'] = [10,12]

def round_up_to_nearest_half(x):
    return math.ceil(x * 2) / 2

In [ ]:
modis_df = load_modis_df(os.path.join(DATA_DIR, 'interim/modis/fire/fire_modis_alaska_2007-2016.pkl'))

# Add local_time column to modis_df
modis_df = modis_df.assign(datetime_local=map(lambda x: du.utc_to_local_time(x[0], x[1], du.round_to_nearest_quarter_hour), zip(modis_df.datetime_utc, modis_df.lon)))

# Load data
sys.path.append(SRC_DIR+'helper')
gfs = load_gfs_weather(os.path.join(DATA_DIR, 'interim/gfs/weather/weather_gfs_alaska_2007-2016_2.pkl'))

cluster_id_5km_10day_df = load_cluster_df(os.path.join(DATA_DIR, 'interim/modis/fire_cluster/fire_cluster_ids_5km_10days_modis_alaska_2007-2016.pkl'))
int_5km_10days_14_df = load_integrated_df(os.path.join(DATA_DIR, 'interim/integrated/fire_weather/fire_weather_integrated_gfs_modis_5km_10days_1400_alaska_2007-2016.pkl'))
int_5km_10days_14_4rain_df = load_integrated_df(os.path.join(DATA_DIR, 'interim/integrated/fire_weather/fire_weather_integrated_gfs_modis_5km_10days_1400_rain-del-4_alaska_2007-2016.pkl'))

# Alaska Fire Prediction

Casey A Graff -- 29 September 2017

## 1. Goal - Regional Smoke Forecasting

## 2. Data
### a) MODIS (Moderate Resolution Imaging Spectroradiometer)

In [ ]:
modis_df[:5]

In [ ]:
ALASKA_BB = get_default_bounding_box()
YEAR_SEL = 2007

mp = make_map(ALASKA_BB)
mp.shadedrelief()

start_date = dt.datetime(YEAR_SEL, 1, 1)
end_date = dt.datetime(YEAR_SEL+1, 1, 1)

sel_df = modis_df[(modis_df.datetime_utc >= start_date) & (modis_df.datetime_utc < end_date)]

_ = mp.scatter(list(sel_df.lon), list(sel_df.lat), 1, latlon=True, marker='o', color='b')
_ = plt.title('All Detections for %d' % YEAR_SEL)

In [ ]:
bounds = [68.5, 69.5, -151.5, -150]
mp = make_map(geo.LatLonBoundingBox(*bounds))
mp.shadedrelief()

sel_df = modis_df[(modis_df.datetime_utc.dt.year==YEAR_SEL) & 
                  (modis_df.lat>bounds[0]) & (modis_df.lat<bounds[1]) &
                  (modis_df.lon>bounds[2]) & (modis_df.lon<bounds[3])]

_ = mp.scatter(list(sel_df.lon), list(sel_df.lat), 5, latlon=True, marker='o', color='b')
_ = plt.title('All Detections for %d (within bounds)' % YEAR_SEL)

### b) GFS (Global Forecast System)

In [5]:
DATA_TYPE = 'temperature'   
DATE_SEL = dt.datetime(2009, 3, 5, 18, tzinfo=pytz.UTC)

mp = make_map(gfs[DATA_TYPE].bounding_box)
mp.shadedrelief()

values = gfs[DATA_TYPE].get_values_for_date(du.DatetimeMeasurement(DATE_SEL))[:,:,0]
lats, lons = gfs[DATA_TYPE].bounding_box.make_grid()

cs = mp.contourf(lons, lats , values, latlon=True, alpha=.6)
cbar = mp.colorbar(cs,location='bottom',pad="5%")

_ = plt.title('%s at %s' % (DATA_TYPE, DATE_SEL))

NameError: name 'gfs' is not defined

## 3. Approach
### a) Global Model
### b) Local/Cluster Model + Ignition Model

In [ ]:
CLUSTER_ID = 249

all_df = cluster_id_5km_10day_df
sel_df = all_df[all_df.cluster_id==CLUSTER_ID]

date_start, date_end = np.min(sel_df.date_local), np.max(sel_df.date_local)
dates = [d for d in du.daterange(date_start, date_end)]

lat_min, lat_max, lon_min, lon_max = np.min(sel_df.lat), np.max(sel_df.lat), np.min(sel_df.lon), np.max(sel_df.lon)
bb = geo.LatLonBoundingBox(round_up_to_nearest_half(lat_min-1), round_up_to_nearest_half(lat_max+1), round_up_to_nearest_half(lon_min-1), round_up_to_nearest_half(lon_max+1))

anim = mapv.animate_map_latlon_all(all_df, CLUSTER_ID, bb, dates, False)

HTML(anim.to_html5_video())

## 4. Results

In [ ]:
plt.rcParams['figure.figsize'] = [8,5]
t_k_arr = np.arange(0, 5)

results = trn.train(int_5km_10days_14_df.copy(), t_k_arr)
plotv.plot_training(results, t_k_arr, metrics.mean_absolute_error)

In [ ]:
plt.rcParams['figure.figsize'] = [8,5]
t_k_arr = np.arange(0, 5)

results = trn.train(int_5km_10days_14_4rain_df.copy(), t_k_arr)
plotv.plot_training(results, t_k_arr, metrics.mean_absolute_error)